# Initialization

In [1]:
import logging, os, sys, time

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

sys.path.insert(0, "/home/pleroy/DEV/posar")
import backprojection as tdbp
from posarmctools import getNotebookLogger
import posarutils.process.disp_PoSAR_img as disp
from posarutils.process.disp_PoSAR_img import OPTt
from posarutils.process.filtering import box_filter

sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia

# this is for the sceneReferencePoint.py module
sys.path.insert(0, "/home/pleroy/DATA/PoSAR-X/PIMA-1")

In [2]:
%matplotlib qt5
eps = np.finfo(float).eps

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
logger = getNotebookLogger(logging.INFO)

In [5]:
libraryFilename = "/home/pleroy/DEV/processing/PoSAR-MC/backprojection/ccpp/libbackprojection/liblibbackprojection.so"

In [ ]:
lib = tdbp.Lib(libraryFilename)

# Load focusing configuration and data

In [ ]:
focusingDir = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/FOCUSING"
# ________________ 11_57_59 ________________
#ini = "2020_06_25_11_57_59_c2_gr_hann.ini"
#ini = "2020_06_25_11_57_59_c2_gr_hann_3.438.ini"
ini = "2020_06_25_11_57_59_c2_gr_noWin_3.438.ini"
#ini = "2020_06_25_11_57_59_c2_gr_hann_3.438_down.ini"
#ini = "2020_06_25_11_57_59_c2_gr_hann_down.ini"
#ini = "2020_06_25_11_57_59_sr_hann.ini"
#ini = "2020_06_25_11_57_59_sr_hann_3.438.ini"
#ini = "2020_06_25_11_57_59_sr_noWin_3.438.ini"
#ini = "2020_06_25_11_57_59_sr_noWin.ini"
#ini = "2020_06_25_11_57_59_c2_sr_hann.ini"
#ini = "2020_06_25_11_57_59_c2_sr_hann_3.438.ini"
#ini = "2020_06_25_11_57_59_c2_sr_noWin_3.438.ini"
#ini = "2020_06_25_11_57_59_c2_sr_noWin.ini"
# ________________ 12_01_28 ________________
#ini = "2020_06_25_12_01_28_c2_gr_hann.ini"
#ini = "2020_06_25_12_01_28_sr_hann.ini"
#ini = "2020_06_25_12_01_28_c2_sr_noWin.ini"
# ________________ 12_04_34 ________________
#ini = "2020_06_25_12_04_34.ini"
# ________________ 12_07_40 ________________
#ini = "2020_06_25_12_07_40_c2_gr_hann.ini"
#ini = "2020_06_25_12_07_40_sr_hann.ini"
#ini = "2020_06_25_12_07_40_sr_noWin.ini"
#ini = "2020_06_25_12_07_40_mower_sr_noWin.ini"
#ini = "2020_06_25_12_07_40_mower_sr_hann.ini"
#ini = "2020_06_25_12_07_40_c2_sr_noWin.ini"
#ini = "2020_06_25_12_07_40_c2_sr_hann.ini"
# ________________ 12_11_37 ________________
#ini = "2020_06_25_12_11_37_c2_gr_hann.ini"
#ini = "2020_06_25_12_11_37_sr_hann.ini"
#ini = "2020_06_25_12_11_37_c2_sr_noWin.ini"

focusing = tdbp.Focusing(os.path.join(focusingDir, ini), tag="Sa", alongTrack=1, pointsFile="ini", corr=True)

## Compute scene elevations

In [ ]:
focusing.rebuildScene()

In [ ]:
fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
ax.set_aspect('equal')
ax.plot(focusing.xa, focusing.ya)
ax.plot(x, y, '.k')
ax.grid()
focusing.addPtsEpsg(ax, ['c0', 'c1', 'c2', 'c3', 'c4'])
focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])
ax.plot(focusing.scene.targetX, focusing.scene.targetY, 'o', label="scene.target")
ax.plot(focusing.track.refX, focusing.track.refY, 'D', label="track.ref")
#ax.plot(focusing.scene.refX, focusing.scene.refY, 'D', label="scene.ref")
#ax.plot(focusing.scene.closestApproach[0], focusing.scene.closestApproach[1], 'o')

# Focalization

In [ ]:
if 0:
    gate = np.zeros(focusing.sa.shape[1])
    slice_ = slice(2000, 3500)
    gate[slice_] = 1.
    sa_aux = np.fft.fft(np.fft.ifft(focusing.sa, axis=1) * gate)
slice_ = slice()

In [ ]:
t = time.time()

focusedImg  = np.zeros( (focusing.scene.nY, focusing.scene.nX), dtype=complex )
print( "focusedImg.shape = {}".format( focusedImg.shape ) )

lib.so.backProjectionOmpGroundRange_LETG(focusing.scene.X.reshape(focusing.scene.X.size), 
                                         focusing.scene.Y.reshape(focusing.scene.Y.size),
                                         focusing.sceneZ.reshape(focusing.sceneZ.size),
                                         focusing.r_over,
                                         focusing.sa.reshape(focusing.sa.size),
                                         focusing.xyz.reshape(focusing.xyz.size), 
                                         focusedImg.reshape(focusedImg.size),
                                         focusing.parameters)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

In [ ]:
myImg = tdbp.Img(focusedImg)
# gr orig min_dB = -51.89, max_dB = 56.58, med_dB = 4.76
# gr corr min_dB = -57.55, max_dB = 53.08, med_dB = 4.83
# sr orig min_dB = -43.25, max_dB = 52.65, med_dB = 7.20
# sr corr min_dB = -45.16, max_dB = 51.15, med_dB = 7.18
# no Win orig min_dB = -32.96, max_dB = 61.69, med_dB = 24.67
# no Win aux min_dB = -53.83, max_dB = 61.70, med_dB = 15.67
# min_dB = -50.82, max_dB = 62.46, med_dB = 15.67

# Plot image

## imshow

In [ ]:
cmap = 'jet'

In [ ]:
left = focusing.conf.xMax
right = focusing.conf.xMin
bottom = focusing.conf.yMax
top = focusing.conf.yMin
extent = [left, right, bottom, top]
vmin = myImg.max_dB - 60 # 60 pour ground range, 40 pour slant range
vmax = myImg.max_dB
#vmin = myImg.med_dB - 15
#vmax = myImg.med_dB + 15

axTitle = f"{focusing.conf.data_date}\n{focusing.conf.groundOrSlant}, {focusing.conf.phi_a_deg:.1f}°"\
            + f", dx {focusing.conf.d_x}, dy {focusing.conf.d_y}"\
            + f", {focusing.conf.window}, corr {focusing.corr}, {slice_}"

if 0:
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
    #im0 = ax0.imshow(myImg.dB, cmap=cmap, extent=extent)
    #im1 = ax1.imshow(myImg.box_dB(3), cmap=cmap, extent=extent)
    #im0 = ax0.imshow(myImg.dB, cmap=cmap, vmin=vmin, vmax=vmax, extent=extent)
    #im1 = ax1.imshow(myImg.dB(3), cmap=cmap, extent=extent)
    dia.addColorBar(im0, ax0)
    dia.addColorBar(im1, ax1)
else:
    fig, ax0 = plt.subplots()
    im0 = ax0.imshow(myImg.dB, cmap=cmap, vmin=vmin, vmax=vmax, extent=extent)
    #im0 = ax0.imshow(myImg.dB, cmap=cmap, extent=extent)
    #im0 = ax0.imshow(myImg.dB, cmap=cmap, vmin=vmin, vmax=vmax)
    dia.addColorBar(im0, ax0)
    ax0.set_title(axTitle)
    ax0.set_aspect('equal')

In [ ]:
plt.figure()
#for idx in range(345, 355, 1):
#    plt.plot(myImg.img_dB.T[idx, :], label=f"{idx}")
idx = 338
toPlot = myImg.img_dB.T[idx, :]
maxVal = np.amax(toPlot)
xAxis = np.arange(focusing.conf.xMax, focusing.conf.xMin, -focusing.conf.d_x)
yAxis = np.arange(focusing.conf.yMin, focusing.conf.yMax, focusing.conf.d_y)
plt.plot(yAxis, toPlot, label=f"coupe à x = {xAxis[idx]:.2f}")
plt.axhline(y=maxVal, color ="green", linestyle ="--") 
plt.axhline(y=maxVal - 3, color ="green", linestyle =":") 
plt.grid()
plt.legend()
plt.xlabel("y [m]")
plt.ylabel("[dB]")

In [ ]:
plt.figure()
#for idx in range(345, 355, 1):
#    plt.plot(myImg.img_dB.T[idx, :], label=f"{idx}")
iy = 105
toPlot = myImg.img_dB.T[:, iy]
maxVal = np.amax(toPlot)
xAxis = np.arange(focusing.conf.xMax, focusing.conf.xMin, -focusing.conf.d_x)
yAxis = np.arange(focusing.conf.yMin, focusing.conf.yMax, focusing.conf.d_y)
plt.plot(xAxis, toPlot, label=f"coupe à y = {yAxis[iy]:.2f}")
plt.axhline(y=maxVal, color ="green", linestyle ="--") 
plt.axhline(y=maxVal - 3, color ="green", linestyle =":") 
plt.grid()
plt.legend()
plt.xlabel("x [m]")
plt.ylabel("[dB]")

## resample

In [ ]:
coef = 4
newImg = signal.resample(focusedImg, focusedImg.shape[0]*coef, axis=0)
newImg = signal.resample(newImg, focusedImg.shape[1]*coef, axis=1)
newMyImg = tdbp.Img(newImg)

In [ ]:
plt.figure()
#for idx in range(345, 355, 1):
#    plt.plot(myImg.img_dB.T[idx, :], label=f"{idx}")
idxAlt = idx * coef
toPlot = newMyImg.img_dB.T[idxAlt, :]
maxVal = np.amax(toPlot)
xAxis = np.arange(focusing.conf.xMax, focusing.conf.xMin, -focusing.conf.d_x / coef)
yAxis = np.arange(focusing.conf.yMin, focusing.conf.yMax, focusing.conf.d_y / coef)
plt.plot(yAxis, toPlot, label=f"coupe à x = {xAxis[idxAlt]:.2f}")
plt.axhline(y = maxVal, color ="green", linestyle ="--") 
plt.axhline(y = maxVal - 3, color ="green", linestyle =":") 
plt.grid()
plt.legend()
plt.xlabel("y [m]")
plt.ylabel("[dB]")

In [ ]:
fig, ax0 = plt.subplots(1, 1, figsize=(10, 5))
im0 = ax0.imshow(newMyImg.dB, cmap='gray', vmin=0, vmax=40)
dia.addColorBar(im0, ax0)

## pcolormesh box_filter

In [ ]:
z = myImg.box_dB(3)

In [ ]:
fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
im = ax.pcolormesh(x, y, z, cmap='jet')
ax.set_aspect('equal')
#ax.plot(focusing.xa, focusing.ya)
plt.colorbar(im)
ax.grid()
#ax.plot(focusing.conf.ptsEpsg['corner'][0], focusing.conf.ptsEpsg['corner'][1], 'og', markeredgecolor = 'black')

In [ ]:
vmin = 0
vmax = 40

fig, ax = plt.subplots(1,1)
x = focusing.scene.X
y = focusing.scene.Y
im = ax.pcolormesh(x, y, z, cmap='jet', vmin=vmin, vmax=vmax)
ax.set_aspect('equal')
ax.plot(focusing.xa, focusing.ya)
plt.colorbar(im)
ax.grid()
#focusing.addPtsEpsg(ax, ['corner'])
#focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])

## Dynamic range

In [ ]:
imgDyn = myImg.dynamicRange(dynRange=70, db=1, size=3)

In [ ]:
fig, ax = plt.subplots(1,1)
im = ax.pcolormesh(x, y, z, cmap=cmap, vmin=-10, vmax=40)
dia.addColorBar(im, ax)
ax.grid()
focusing.addPtsEpsg(ax, ['c2'])
focusing.addPtsEpsg(ax, ['h0', 'h1', 'h2', 'h3'])

# Build GeoTIFF

In [ ]:
tiffName = os.path.join(focusing.out_dir, focusing.conf.name + ".tif")

In [ ]:
myTiff = tdbp.Tiff(tiffName, focusing, z, cmap='gray')

# Save focused image

In [ ]:
myImg.save(focusing)

In [ ]:
myImg.saveAlt(focusing)

In [ ]:
file = "/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/FOCUSING/2020_06_25_11_57_59/2020_06_25_11_57_59_sr_hann_3.438.data"
otherImg = np.fromfile(file, dtype='complex64').reshape(480, 700)